In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import zipfile
with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip') as existing_zip:
    existing_zip.extractall()
with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip') as existing_zip:
    existing_zip.extractall()

In [ ]:
cd train

In [ ]:
mkdir cats

In [ ]:
mkdir dogs

In [ ]:
import shutil
import glob
import os

def move_glob(dst_path, pathname, recursive=True):
    for p in glob.glob(pathname, recursive=recursive):
        shutil.move(p, dst_path)

move_glob('./dogs', 'dog*.jpg')
move_glob('./cats', 'cat*.jpg')

In [ ]:
cd ../

In [ ]:
mkdir valid

In [ ]:
cd valid

In [ ]:
mkdir dogs

In [ ]:
mkdir cats

In [ ]:
cd ..

In [ ]:
import cv2
import numpy as np

def resize(src_file, dst_file, width, height):
    """
    画像ファイルを与えられたサイズにサイズ変換する関数
    アス比は固定、左上に寄せる、余った部分はゼロで埋める

    Returns
    -------
    ret_scale : float
    拡大/縮小した倍率
    """

    src_img = cv2.imread(src_file)
    h, w, c = src_img.shape
    # dst_img = cv2.resize(src_img, dsize=(width, height))

    # アス比固定, padding
    scale_w = width / w
    scale_h = height / h

    ret_scale = 1.0
    # Down Convert
    if(scale_w < 1.0 or scale_h < 1.0):
        if(scale_w < scale_h): 
            resize_img = cv2.resize(src_img, dsize=None, fx=scale_w, fy=scale_w, interpolation = cv2.INTER_AREA)
            ret_scale = scale_w
        else:
            resize_img = cv2.resize(src_img, dsize=None, fx=scale_h, fy=scale_h, interpolation = cv2.INTER_AREA)
            ret_scale = scale_h
    else:
        resize_img = src_img

    # dst_img 生成
    dst_img = np.zeros((height, width, 3), dtype = np.uint8)

    # dst_imgにresize_imgを合成
    top = 0
    left = 0
    #dst_img[top:height + top, left:width + left] = resize_img
    h, w, c = resize_img.shape
    dst_img[0:h, 0:w] = resize_img

    cv2.imwrite(dst_file, dst_img)

    return ret_scale

In [ ]:
for num in range(0, 12500):
    resize('train/dogs/dog.' + str(num) + '.jpg', 'train/dogs/dog.' + str(num) + '.jpg', 216, 216)
    resize('train/cats/cat.' + str(num) + '.jpg', 'train/cats/cat.' + str(num) + '.jpg', 216, 216)

for num in range(2500):
    move_glob('valid/cats', 'train/cats/cat.'+ str(num) +'.jpg')
    move_glob('valid/dogs', 'train/dogs/dog.'+ str(num) +'.jpg')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os
import datetime
import tensorflow as tf

In [ ]:
train_data_dir = 'train'
valid_data_dir = 'valid'
bs = 128

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255, rotation_range=90, horizontal_flip=True, channel_shift_range=50.)
valid_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    directory=train_data_dir, target_size=(216, 216), batch_size=bs, class_mode="binary")
valid_generator = valid_datagen.flow_from_directory(
    directory=valid_data_dir, target_size=(216, 216), batch_size=bs, class_mode="binary")

In [ ]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(5, 5),
          activation="relu", kernel_regularizer=regularizers.l2(0.0001), input_shape=(216, 216, 3)))

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(5, 5),
          kernel_regularizer=regularizers.l2(0.0001), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(5, 5),
          kernel_regularizer=regularizers.l2(0.0001), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(5, 5),
          kernel_regularizer=regularizers.l2(0.0001), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

opt = optimizers.Adam(learning_rate=0.0001)
model.compile(loss="binary_crossentropy",
              optimizer=opt, metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_generator, 
                    epochs=80, 
                    verbose=1, 
                    validation_data=(valid_generator), 
                    steps_per_epoch=20000 / bs, 
                    validation_steps=5000/bs)

In [ ]:
model.save("model.h5")

In [ ]:
import matplotlib.pyplot as plt

loss = history.history["loss"]
val_loss = history.history["val_loss"]

learning_count = len(loss)+1

plt.plot(range(1,learning_count),loss,marker = "+",label = "loss")
plt.plot(range(1,learning_count),val_loss,marker = "+",label = "val_loss")
plt.legend(loc = "best",fontsize = 10)
plt.xlabel("learning_count")
plt.ylabel("loss")
plt.show()

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]

learning_count = len(loss)+1

plt.plot(range(1,learning_count),accuracy,marker = "+",label = "accuracy")
plt.plot(range(1,learning_count),val_accuracy,marker = "+",label = "val_accuracy")
plt.legend(loc = "best",fontsize = 10)
plt.xlabel("learning_count")
plt.ylabel("accuracy")
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

In [ ]:
new_model = load_model('model.h5')

In [ ]:
result = []

for num in range(1, 12501):
    # 予測したい画像のパス
    TESTPATH = "test/" + str(num) + ".jpg"

    # 予測モデルに入力できるように画像を配列に落とし込む
    img = image.load_img(TESTPATH, target_size=(216, 216))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0
    # 予測
    result_predict = new_model.predict(x)
    # print(result_predict)
    result.append(result_predict)

In [ ]:
result_num = []

for num in range(1, 12501):
    result_num.append(num)

In [ ]:
df_res = pd.DataFrame([result])
df_res = df_res.T
for col in df_res:
    df_res[col] = df_res[col].astype(
        str).str.replace("[", "").str.replace("]", "")
df = pd.read_csv('../input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
df['label'] = df_res

In [ ]:
df

In [ ]:
df.to_csv("submission.csv", index=False)